In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [12]:
loader = PyPDFLoader("yolov11.pdf")
data = loader.load()

In [13]:
data

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-24T00:37:53+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-24T00:37:53+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'yolov11.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='YOLO V11: A N OVERVIEW OF THE KEY ARCHITECTURAL\nENHANCEMENTS\nRahima Khanam* and Muhammad Hussain\nDepartment of Computer Science, Huddersfield University, Queensgate, Huddersfield HD1 3DH, UK;\n*Correspondence: rahima.khanam@hud.ac.uk;\nOctober 24, 2024\nABSTRACT\nThis study presents an architectural analysis of YOLOv11, the latest iteration in the YOLO (You Only\nLook Once) series of object detection models. We examine the models architectural innovations,\nincluding the introduction of the C3k2 (Cross Stage Partial with kernel size 2) block, SPP

In [14]:
len(data)

9

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total number of documents: ", len(docs))

Total number of documents:  40


In [10]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

ModuleNotFoundError: No module named 'langchain_chroma'

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

In [ ]:
vectorspace = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
retriever = vectorspace.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("What is new in yolov11?")

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name="Llama3-8b-8192", temperature=0, max_tokens=500)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [18]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is new in YOLOv11?"})
print(response["answer"])